In [1]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = os.getenv('PINECONE_API_KEY')
os.environ['LANGCHAIN_TRACING_V2']='true'
os.environ['LANGCHAIN_API_KEY']=os.getenv('LANGCHAIN_API_KEY')

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

c:\Users\silar\OneDrive\Desktop\minor_class_work\qpgen\githubcopilot\.venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
import time

index_name = "hybrid"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [3]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [4]:
# Load BM25Retriever from a file
import pickle
with open("bm25_model.pkl", "rb") as f:
    bm25_retriever = pickle.load(f)

print("BM25 retriever loaded.")

BM25 retriever loaded.


In [5]:
from langchain.vectorstores import Pinecone
name =" " # agent to find the correct namespace from the index
vector_store = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings)

In [6]:
# setting up retreiver
retriever = vector_store.as_retriever(
        search_type="mmr",
        search_kwargs={'k': 20, 'fetch_k': 50}
    )
retriever

#2
retriever2 = vector_store.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={'score_threshold': 0.25}
    )
retriever2

VectorStoreRetriever(tags=['Pinecone', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.pinecone.Pinecone object at 0x000001BF58BD9650>, search_type='similarity_score_threshold', search_kwargs={'score_threshold': 0.25})

In [7]:
from langchain.retrievers import EnsembleRetriever

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever],
    weights=[0.5, 0]  # Adjust weights as needed
)


ensemble_retriever2 = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever2],
    weights=[0.5, 0.5]  # Adjust weights as needed
)

In [8]:
from config import p

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.5)
llm

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(template=p)
print(prompt)



input_variables=['context', 'difficulty', 'input', 'number', 'other', 'qn_type', 'special_mention'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'difficulty', 'input', 'number', 'other', 'qn_type', 'special_mention'], input_types={}, partial_variables={}, template="You are a teaching assistant, and your task is to make a question paper for the students for a certain class.\nThe question paper can have multiple types of questions like MCQs, short answer questions, long answer questions, numerical questions, graphs, descriptive questions, etc.\nYou have to design a question paper for the students such that it helps them understand the concepts well.\nHere are some of the things to keep in mind:\n1. The question paper should be based on the chapter selected.\n2. The question paper should be based on the learning objectives provided (if any).\n3. The question paper should be based on the difficulty level selected

In [9]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.5)
llm

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(template=p)
print(prompt)


input_variables=['context', 'difficulty', 'input', 'number', 'other', 'qn_type', 'special_mention'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'difficulty', 'input', 'number', 'other', 'qn_type', 'special_mention'], input_types={}, partial_variables={}, template="You are a teaching assistant, and your task is to make a question paper for the students for a certain class.\nThe question paper can have multiple types of questions like MCQs, short answer questions, long answer questions, numerical questions, graphs, descriptive questions, etc.\nYou have to design a question paper for the students such that it helps them understand the concepts well.\nHere are some of the things to keep in mind:\n1. The question paper should be based on the chapter selected.\n2. The question paper should be based on the learning objectives provided (if any).\n3. The question paper should be based on the difficulty level selected

In [10]:
def prompt_template(input,
           qn_type=["multiple choice", "short answer", "long answer", "numericals"],
           qn_type_count = [5,2,2,5],
           difficulty='medium',
           number='10',
           special_mention='none',
           other='none'):
    

    # type of qns:

    dictionary = dict(zip(qn_type, qn_type_count))
    question_type = ''
    for x in dictionary:
        q = x
        qn = dictionary[q]
        question_type = question_type + f"{q}s: {qn}, "
        

    # making final prompt
    providing_data = {'input':input,
                    'qn_type':question_type,
                     'difficulty':difficulty,
                     'number':number,
                     'special_mention':special_mention,
                     'other':other
                     }
    return providing_data

In [11]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
)

In [12]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(ensemble_retriever,document_chain)

In [13]:
input = "What are ions?"
result = prompt_template(
    input=input,
    #qn_type=["mcq", "short_answer", "long_answer", "numerical"],
    #qn_type='numerical',
    qn_type_count=[5,5,0,10],
    difficulty="medium",
    number="2",
    special_mention="Focus on equation based questions",
    other="Try to cover formulas from these topics"
)
r = result


In [14]:
response=retrieval_chain.invoke(r) #ai agent to make prompts better
response

{'input': 'What are ions?',
 'qn_type': 'multiple choices: 5, short answers: 5, long answers: 0, numericalss: 10, ',
 'difficulty': 'medium',
 'number': '2',
 'special_mention': 'Focus on equation based questions',
 'other': 'Try to cover formulas from these topics',
 'context': [Document(metadata={}, page_content='1.BASIC CONCEPTS, SYMBOLS AND FORMULA \n \n5 \n SANKALP GROUP OF SCHOOLS \n  \n5. Define the term \'Atomic Weight\'. Give two examples in which elements have fractional \natomic weights. \n6. Write the value of 1 atomic mass unit (1 amu) in kg. \n7. Write the definition of the term \'mole\'. \n8. How many electrons are there in 1.5 mole of electrons. \n9. Name the S.I. unit of mass. \n10. Name the S.I. unit of density of an object. \n11. What is the length of an insect in meter which is 5.3pm ? \n12. Convert 30.5∘C into temperature in Fahrenheit scale. \n13. What is the S.I. unit of Force? \n14. What is the S.I. unit of power? \nHint. Power is energy per unit time. In SI uni

In [15]:
print(response['answer'])

# Questions

### Section 1: Multiple Choice Questions (MCQs)
1. Which of the following best defines an ion?
   a) A neutral atom  
   b) An atom that has gained or lost electrons  
   c) A molecule made of two or more atoms  
   d) A type of compound  

2. What is the charge of a sodium ion (Na⁺)?
   a) Negative  
   b) Positive  
   c) Neutral  
   d) Variable  

3. Which of the following ions is a common component of table salt?
   a) Na⁺  
   b) Cl⁻  
   c) Ca²⁺  
   d) Both a and b  

4. Anions are formed when an atom:
   a) Loses electrons  
   b) Gains electrons  
   c) Gains protons  
   d) Loses neutrons  

5. The ion with the chemical formula SO₄²⁻ is known as:
   a) Sulfate  
   b) Sulfite  
   c) Sulfur  
   d) Sulfide  

### Section 2: Short Answer Questions
1. Define the term 'ion' and explain how ions are formed.
2. What is the significance of ions in chemical reactions? Provide an example.
3. Describe the difference between cations and anions.
4. Explain how ionic compou